In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
BATCH_SIZE = 523  	# batch_size即每批训练的样本数量
epochs = 20			# 循环次数
DEVICE=torch.device("cuda" if torch.cuda.is_available() else "cpu")    #判断是否能在GPU上进行运算


In [28]:
print(DEVICE)
print(torch.backends.cudnn.enabled)
# torch.backends.cudnn.enabled=False
torch.backends.cudnn.enabled=True
print(torch.backends.cudnn.enabled)


cuda
False
True


In [29]:
train_loader = torch.utils.data.DataLoader(                 # vision.utils : 用于把形似 (3 x H x W) 的张量保存到硬盘中，给一个mini-batch的图像可以产生一个图像格网。
        datasets.MNIST('data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),       # 图像转化为Tensor
                           transforms.Normalize((0.1307,), (0.3081,))       # 标准化
                       ])),
        batch_size=BATCH_SIZE, shuffle=True)            # shuffle() 方法将序列的所有元素随机排序


In [30]:
test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('data', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=BATCH_SIZE, shuffle=True)            # shuffle() 方法将序列的所有元素随机排序


In [31]:
class Net(nn.Module): # 继承model
    def __init__(self):
        super().__init__()
        # 28x28
        self.conv1=nn.Conv2d(1,12,5)         # 12, 24x24
        self.conv2=nn.Conv2d(12, 20,3)       #20, 10x10
        self.conv3=nn.Conv2d(20, 40,3)       #20, 10x10
        self.batchnorm2d = nn.BatchNorm2d(40)
        self.fc1=nn.Linear(40*8*8, 500)
        self.fc2=nn.Linear(500, 10)
    def forward(self, x):      #网络传播结构
        in_size=x.size(0)# in_size 为 batch_size（一个batch中的Sample数）
        # 卷积层 -> relu -> 最大池化
        out = self.conv1(x)     # 24
        out = F.relu(out)
        out = F.max_pool2d(out, 2, 2)  # 12
        out = self.conv2(out)  # 10
        out = F.relu(out)
        out = self.conv3(out)
        out = F.relu(out)
        out =  self.batchnorm2d(out)
        out = out.view(in_size, -1)    # view()函数作用是将一个多行的Tensor,拼接成行。
        # 输出前的预处理
        out = self.fc1(out)
        out = F.relu(out)
        out = self.fc2(out)
        # softmax
        out = F.log_softmax(out, dim=1)
        # 返回值 out
        return out


In [32]:
model = Net().to(DEVICE)
optimizer = optim.Adam(model.parameters())


In [33]:
def train(model, device, train_loader, optimizer, epoch):
    model.eval()
    for  epoch_i in range(epoch+1):
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data, target      # CPU转GPU
            data, target = data.to(device), target.to(device)   # CPU -> GPU

            optimizer.zero_grad()               # 优化器清零
            output = model(data)                # 由model，计算输出值
            loss = F.nll_loss(output, target)   # 计算损失函数loss
            loss.backward()                     # loss反向传播
            optimizer.step()                    # 优化器优化
            if(batch_idx+1)%30 == 0:            # 输出结果
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch_i, batch_idx * len(data), len(train_loader.dataset),
                    100. * batch_idx / len(train_loader), loss.item()))


In [34]:
def test(model, device, test_loader):
    test_loss = 0                           # 损失函数初始化为0
    correct = 0                             # correct 计数分类正确的数目
    with torch.no_grad():           
        for data, target in test_loader:    # 遍历所有的data和target
            data, target = data.to(device), target.to(device)   # CPU -> GPU
            output = model(data)            # output为预测值，由model计算出
            test_loss += F.nll_loss(output, target, reduction='sum').item()     ### 将一批的损失相加
            pred = output.max(1, keepdim=True)[1]       ### 找到概率最大的下标
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)   # 总损失除数据集总数
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


In [35]:
import time
old=time.time()
train(model, DEVICE, train_loader, optimizer, 1)
print(time.time()-old)
test(model, DEVICE, test_loader)


Train Epoch: 0 [15167/60000 (25%)]	Loss: 0.354718
Train Epoch: 0 [30857/60000 (51%)]	Loss: 0.166337
Train Epoch: 0 [46547/60000 (77%)]	Loss: 0.143088
Train Epoch: 1 [15167/60000 (25%)]	Loss: 0.077669
Train Epoch: 1 [30857/60000 (51%)]	Loss: 0.086047
Train Epoch: 1 [46547/60000 (77%)]	Loss: 0.090376
7.203686714172363

Test set: Average loss: 0.0460, Accuracy: 9846/10000 (98%)

